In [26]:
#coding: utf-8
# citations: https://pythonmana.com/2021/03/20210329161147051K.html
# citation: https://stackoverflow.com/questions/43637211/retrieve-document-content-with-document-structure-with-python-docx

# Import Libraries
!pip install docx
from docx import Document
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import pandas as pd
import numpy as np
import re
from os.path import exists

########################################################################
# USER INPUTS
########################################################################
# Replace File Path
path = 'C:/Users/jstockham/OneDrive - American Institutes for Research in the Behavioral Sciences/JSmith/PYTHON_CODE_2022/table_automate_jess/general_table_automate/'

# Word Document Name (If your Word doc is tableshells.docx, word_doc = 'tableshells.docx')
word_doc = 'tableshells.docx'

# Excel Document Name (If your Excel doc is rawdata.xlsx, excel_doc = 'rawdata.xlsx')
excel_doc = 'rawdata.xlsx'

# Crosswalk Excel Document Name (If your Excel doc is crosswalk.xlsx, crosswalk_doc = 'crosswalk.xlsx')
crosswalk_doc = 'crosswalk.xlsx'


            

In [30]:
########################################################################
# Load files
########################################################################

# 1. Load raw data into Pandas Dataframe
tables_data = pd.read_excel(excel_doc)
tables_data.head()

# 3. Load Crosswalk data into Pandas Dataframe

# 2. Load Word Document into a docx file object
document = Document(word_doc)




# # # Obtain table shells
# tables = docx.tables
# table=tables[0]

# print(docx.tables)


# # In[9]:


# print(len(table.rows))
# print(len(table.columns))




,Table 1,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Category,Participant,Pre-Participant,Total
1,Google/Online,0.028,0.012,0.026
2,Faculty,0.603,0.612,0.604
3,Outreach Event,0.095,0.106,0.097
4,Word-of-Mouth,0.127,0.082,0.12


In [15]:
# Loop through each table in the Word file

# 1. Identify Paragraph/Table pairs
def iter_block_items(parent):
    """
    Generate a reference to each paragraph and table child within *parent*,
    in document order. Each returned value is an instance of either Table or
    Paragraph. *parent* would most commonly be a reference to a main
    Document object, but also works for a _Cell object, which itself can
    contain paragraphs and tables.
    """
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)


for block in iter_block_items(document):
    #print('found one')
    print(block.text if isinstance(block, Paragraph) else '<table>')


    #2. Look for the block.text in the Master List dataframe

    #3. If found in Master List, load in the appropriate sheet in Excel file into a table dataframe (table_df). 1 table per sheet

    #4. Loop through the dataframe table_df and populate the rows of the Table Object
    # How do you interact with that specific Table Object?


Table 1
<table>

Table 2
<table>

